<a href="https://colab.research.google.com/github/rebeccajohnson88/ar_mentalhealthproj/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyLDAvis

In [ ]:
%pip install tweepy -U  

In [ ]:
#import relevant packages 
import tweepy
import pandas as pd
import nltk 
nltk.download('stopwords')
from nltk.stem.porter import *



In [3]:
import gensim
from gensim import corpora

In [7]:
nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize, word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [87]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [8]:
stopwords_list = stopwords.words('english') 
porter = PorterStemmer()

In [9]:
#initialize client 
client = tweepy.Client(bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJe%2BZgEAAAAAYrT%2BKy9PiNrkWJ3KyKE6HIRrZBM%3DfMh2oybMX46FfAGMAoqdm6tkTAaLvRPBWsbvS6l4lPIJ9Up6qr')

In [90]:
q = "(#BTS) lang:en"

tweet_resp = client.search_recent_tweets(query = q,
                                         max_results = 100, 
                                         tweet_fields = ['text','in_reply_to_user_id'],
                                         user_fields = ['name', 'description'],
                                         expansions = ['author_id'])

In [115]:
[tweet['description'].replace('\n','') for tweet in tweet_resp.includes['users']][:5]

['💜🧸 Where is my Angel? 🧸💜',
 'bts paved the way.',
 'you got the best of me, bts ! ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏͏ ͏ ͏ ͏ ͏',
 '💜LOVE💜(YOUR)NATURE😊🌍🌏!#GetItLetItRoll\U0001f9c8!+21 please(🔞)Artwork credits:_Icon- @bebe_sope /_banner- @bebe_sope 💜',
 'K-pop fan ,bts']

In [114]:
[user['name'] for user in tweet_resp.includes['users']][:5]

['ᗷᒪᑌᗴ & ᘜᖇᗴY 💜',
 'namo do bangtan | ⁷ 💭',
 'babi⁷',
 'Sarafol💜🕺🕴️💃\U0001f9c8🌌',
 'Bts Fan']

In [113]:
tweet_resp.includes['users'][:5]

[<User id=338606125 name=ᗷᒪᑌᗴ & ᘜᖇᗴY 💜 username=TigerBear_1>,
 <User id=1467558253320093696 name=namo do bangtan | ⁷ 💭 username=straornt>,
 <User id=1344008251806019587 name=babi⁷ username=pjmwhy>,
 <User id=700525549504974848 name=Sarafol💜🕺🕴️💃🧈🌌 username=sarafoool>,
 <User id=1438000695681056771 name=Bts Fan username=BtsFan981>]

In [ ]:
tweet_resp.data

In [95]:
# Build DataFrame using a tweet's keys as the columns 
tweet_df = pd.DataFrame(tweet_resp.data)

# Print head of DataFrame
tweet_df.head()


,author_id,id,in_reply_to_user_id,text
0,338606125,1502453267925811207,NaN,RT @TigerBear_1: @TOP100KPOP I vote #BTSV for ...
1,1467558253320093696,1502453266428354561,NaN,kim taehyung Listen to #BTS_Butter the best se...
2,1344008251806019587,1502453264863928321,NaN,RT @sugacrap: I'm completely in love with #Sta...
3,700525549504974848,1502453263672778754,NaN,RT @ZER0XBLACK: 220311 RM Instagram \nhttps://...
4,1438000695681056771,1502453263425163265,NaN,RT @btsretweetsTS50: I vote for #Butter for #B...


In [96]:
#User dataframe 
user_df = pd.DataFrame(tweet_resp.includes['users'])
user_df['author_id'] = user_df['id']

#Print head 
user_df.head()

,description,id,name,username,author_id
0,💜🧸 Where is my Angel? 🧸💜,338606125,ᗷᒪᑌᗴ & ᘜᖇᗴY 💜,TigerBear_1,338606125
1,bts paved the way.,1467558253320093696,namo do bangtan | ⁷ 💭,straornt,1467558253320093696
2,"you got the best of me, bts ! ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ...",1344008251806019587,babi⁷,pjmwhy,1344008251806019587
3,💜LOVE💜(YOUR)NATURE😊🌍🌏!\n#GetItLetItRoll🧈!\n+21...,700525549504974848,Sarafol💜🕺🕴️💃🧈🌌,sarafoool,700525549504974848
4,"K-pop fan ,bts",1438000695681056771,Bts Fan,BtsFan981,1438000695681056771


In [97]:
BTS_tweets = tweet_df.merge(user_df, on='author_id')

In [98]:
BTS_tweets['text'] = BTS_tweets['text'].str.replace('\n','')
BTS_tweets['description'] = BTS_tweets['description'].str.replace('\n', '')

In [99]:
BTS_tweets['description'] = BTS_tweets['description'].str.replace('.', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [100]:
corpus_lower = BTS_tweets['description'].str.lower().to_list()

In [101]:
def cleaning_function(string): 
  string_nostops = [word for word in wordpunct_tokenize(string) if word not in stopwords_list]
  string_cleaned = [porter.stem(token) for token in string_nostops if token.isalpha() and len(token) > 1]
  return string_cleaned

In [102]:
corpus_cleaned = [cleaning_function(desc) for desc in corpus_lower]

In [103]:
corpus_dict = corpora.Dictionary(corpus_cleaned)

In [104]:
first_entries = {k: corpus_dict[k] for k in corpus_dict.keys()[:6]}

In [105]:
lower_bound = round(BTS_tweets.description.shape[0]*0.05)
upper_bound = round(BTS_tweets.description.shape[0]*0.95)

In [106]:
corpus_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)

In [107]:
corpus_fromdict = [corpus_dict.doc2bow(one_text) for one_text in corpus_cleaned]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 2, id2word=corpus_dict, 
                                         passes = 2, alpha = 'auto',
                                        per_word_topics = True)


In [109]:
topics = lda_model.print_topics(num_words = 10)
for topic in topics:
    print(topic)

(0, '0.189*"kpop" + 0.113*"armi" + 0.093*"taehyung" + 0.092*"hater" + 0.091*"hous" + 0.091*"anti" + 0.088*"bt" + 0.052*"bangtan" + 0.040*"pop" + 0.039*"account"')
(1, '0.283*"bt" + 0.179*"love" + 0.104*"armi" + 0.085*"account" + 0.078*"way" + 0.067*"fan" + 0.053*"wreck" + 0.051*"pop" + 0.044*"bangtan" + 0.018*"taehyung"')


In [110]:
topic_probability = [lda_model.get_document_topics(item) for item in corpus_fromdict]

corpus_cleaned[0:5]
topic_probability[0:5]

[[(0, 0.4579789), (1, 0.5420211)],
 [(0, 0.4579789), (1, 0.5420211)],
 [(0, 0.4579789), (1, 0.5420211)],
 [(0, 0.20128044), (1, 0.7987196)],
 [(0, 0.2012824), (1, 0.7987176)]]

In [111]:
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(lda_model, corpus_fromdict, corpus_dict)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
